<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary 代码 for 这个 <一个 href="http://mng.bz/orYv">构建 一个 大语言模型 From Scratch</一个> book by <一个 href="https://sebastianraschka.com">Sebastian Raschka</一个><br>
<br>代码 repository: <一个 href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</一个>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<一个 href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></一个>
</td>
</tr>
</table>


# 第 2: Working with Text Data

Packages 那个 are being used in 这个 笔记本:

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.5.1
tiktoken version: 0.7.0


- 这个 第 covers data preparation 和 sampling to 获取 输入 data "ready" for 这个 大语言模型

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 Understanding word embeddings

- No 代码 in 这个 section

- 那里 are many forms of embeddings; 我们 focus on text embeddings in 这个 book

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- LLMs work with embeddings in high-dimensional spaces (i.e., thousands of dimensions)
- Since 我们 can't 可视化 such high-dimensional spaces (我们 humans think in 1, 2, 或者 3 dimensions), 这个 figure below illustrates 一个 2-dimensional 嵌入 space

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 Tokenizing text

- In 这个 section, 我们 tokenize text, 哪个 means breaking text into smaller units, such as individual words 和 punctuation characters

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- 加载 raw text 我们 want to work with
- [这个 Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) is 一个 public domain short story

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

- (如果 你 encounter 一个 `ssl.SSLCertVerificationError` 当 executing 这个 previous 代码 cell, 它 might be due to using 一个 outdated Python version; 你 can find [more information 这里 on GitHub](https://github.com/rasbt/LLMs-from-scratch/pull/403))

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 这个 goal is to tokenize 和 embed 这个 text for 一个 大语言模型
- 让我们 develop 一个 simple 分词器 based on some simple sample text 那个 我们 can 然后 later 应用 to 这个 text above
- 这个 following regular expression will split on whitespaces

In [4]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 我们 don't only want to split on whitespaces 但是 also commas 和 periods, so 让我们 修改 这个 regular expression to do 那个 as well

In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- As 我们 can see, 这个 creates empty strings, 让我们 移除 them

In [6]:
# Strip whitespace from each item 和 然后 filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 这个 looks pretty good, 但是 让我们 also handle other types of punctuation, such as periods, question marks, 和 so on

In [7]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 这个 is pretty good, 和 我们 are 现在 ready to 应用 这个 tokenization to 这个 raw text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 让我们 计算 这个 total number of tokens

In [9]:
print(len(preprocessed))

4690


## 2.3 Converting tokens into 词元 IDs

- 接下来, 我们 转换 这个 text tokens into 词元 IDs 那个 我们 can 处理 via 嵌入 layers later

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- From these tokens, 我们 can 现在 构建 一个 vocabulary 那个 consists of all 这个 unique tokens

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- Below are 这个 首先 50 entries in 这个 vocabulary:

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- Below, 我们 illustrate 这个 tokenization of 一个 short sample text using 一个 small vocabulary:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- Putting 它 现在 all together into 一个 分词器 类

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before 这个 specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- 这个 `encode` 函数 turns text into 词元 IDs
- 这个 `decode` 函数 turns 词元 IDs back into text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- 我们 can 使用 这个 分词器 to encode (那个 is, tokenize) texts into integers
- These integers can 然后 be embedded (later) as 输入 of/for 这个 大语言模型

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"它's 这个 last he painted, 你 know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 我们 can decode 这个 integers back into text

In [15]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [16]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 Adding special context tokens

- 它's useful to 添加 some "special" tokens for unknown words 和 to denote 这个 结束 of 一个 text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- Some tokenizers 使用 special tokens to help 这个 大语言模型 with additional context
- Some of these special tokens are
  - `[BOS]` (beginning of sequence) marks 这个 beginning of text
  - `[EOS]` (结束 of sequence) marks 哪里 这个 text ends (这个 is usually used to concatenate multiple unrelated texts, e.g., two different Wikipedia articles 或者 two different books, 和 so on)
  - `[PAD]` (padding) 如果 我们 train LLMs with 一个 批量大小 greater than 1 (我们 may include multiple texts with different lengths; with 这个 padding 词元 我们 pad 这个 shorter texts to 这个 longest length so 那个 all texts have 一个 equal length)
- `[UNK]` to represent words 那个 are not included in 这个 vocabulary

- Note 那个 GPT-2 does not need any of these tokens mentioned above 但是 only uses 一个 `<|endoftext|>` 词元 to reduce complexity
- 这个 `<|endoftext|>` is analogous to 这个 `[EOS]` 词元 mentioned above
- GPT also uses 这个 `<|endoftext|>` for padding (since 我们 typically 使用 一个 mask 当 训练 on batched inputs, 我们 would not attend padded tokens anyways, so 它 does not matter 什么 these tokens are)
- GPT-2 does not 使用 一个 `<UNK>` 词元 for out-of-vocabulary words; instead, GPT-2 uses 一个 byte-pair encoding (BPE) 分词器, 哪个 breaks down words into subword units 哪个 我们 will discuss in 一个 later section



- 我们 使用 这个 `<|endoftext|>` tokens between two independent sources of text:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- 让我们 see 什么 happens 如果 我们 tokenize 这个 following text:

In [17]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 这个 above produces 一个 error because 这个 word "Hello" is not contained in 这个 vocabulary
- To deal with such cases, 我们 can 添加 special tokens like `"<|unk|>"` to 这个 vocabulary to represent unknown words
- Since 我们 are already extending 这个 vocabulary, 让我们 添加 another 词元 called `"<|endoftext|>"` 哪个 is used in GPT-2 训练 to denote 这个 结束 of 一个 text (和 它's also used between concatenated text, like 如果 our 训练 datasets consists of multiple articles, books, etc.)

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 我们 also need to adjust 这个 分词器 accordingly so 那个 它 knows 当 和 如何 to 使用 这个 new `<unk>` 词元

In [21]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before 这个 specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

让我们 try to tokenize text with 这个 modified 分词器:

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 BytePair encoding

- GPT-2 used BytePair encoding (BPE) as its 分词器
- 它 allows 这个 模型 to break down words 那个 aren't in its predefined vocabulary into smaller subword units 或者 even individual characters, enabling 它 to handle out-of-vocabulary words
- For instance, 如果 GPT-2's vocabulary doesn't have 这个 word "unfamiliarword," 它 might tokenize 它 as ["unfam", "iliar", "word"] 或者 some other subword breakdown, depending on its trained BPE merges
- 这个 original BPE 分词器 can be found 这里: [https://github.com/openai/GPT-2/blob/master/src/encoder.py](https://github.com/openai/GPT-2/blob/master/src/encoder.py)
- In 这个 第, 我们 are using 这个 BPE 分词器 from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) 库, 哪个 implements its core algorithms in Rust to improve computational 性能
- I created 一个 笔记本 in 这个 [./bytepair_encoder](../02_bonus_bytepair-encoder) 那个 compares these two implementations side-by-side (tiktoken was about 5x faster on 这个 sample text)

In [25]:
# pip 安装 tiktoken

In [26]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [27]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE tokenizers break down unknown words into subwords 和 individual characters:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 Data sampling with 一个 sliding window

- 我们 train LLMs to 生成 one word at 一个 time, so 我们 want to prepare 这个 训练 data accordingly 哪里 这个 接下来 word in 一个 sequence represents 这个 目标 to predict:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- For each text chunk, 我们 want 这个 inputs 和 targets
- Since 我们 want 这个 模型 to predict 这个 接下来 word, 这个 targets are 这个 inputs shifted by one position to 这个 right

In [31]:
enc_sample = enc_text[50:]

In [32]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- One by one, 这个 预测 would look like as follows:

In [33]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 我们 will take care of 这个 接下来-word 预测 in 一个 later 第 after 我们 covered 这个 注意力机制 mechanism
- For 现在, 我们 实现 一个 simple 数据加载器 那个 iterates over 这个 输入 数据集 和 returns 这个 inputs 和 targets shifted by one

- 安装 和 导入 PyTorch (see 附录 一个 for 安装 tips)

In [35]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.5.1


- 我们 使用 一个 sliding window approach, changing 这个 position by +1:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- 创建 数据集 和 dataloader 那个 extract chunks from 这个 输入 text 数据集

In [36]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize 这个 entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # 使用 一个 sliding window to chunk 这个 book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [37]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # 初始化 这个 分词器
    tokenizer = tiktoken.get_encoding("gpt2")

    # 创建 数据集
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 创建 dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- 让我们 测试 这个 dataloader with 一个 批量大小 of 1 for 一个 大语言模型 with 一个 context size of 4:

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 一个 示例 using stride equal to 这个 context length (这里: 4) as shown below:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- 我们 can also 创建 batched outputs
- Note 那个 我们 increase 这个 stride 这里 so 那个 我们 don't have overlaps between 这个 batches, since more overlap could lead to increased overfitting

In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 Creating 词元 embeddings

- 这个 data is already almost ready for 一个 大语言模型
- 但是 lastly let us embed 这个 tokens in 一个 continuous vector representation using 一个 嵌入 层
- Usually, these 嵌入 layers are part of 这个 大语言模型 itself 和 are updated (trained) during 模型 训练

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- Suppose 我们 have 这个 following four 输入 examples with 输入 ids 2, 3, 5, 和 1 (after tokenization):

In [42]:
input_ids = torch.tensor([2, 3, 5, 1])

- For 这个 sake of simplicity, suppose 我们 have 一个 small vocabulary of only 6 words 和 我们 want to 创建 embeddings of size 3:

In [43]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 这个 would result in 一个 6x3 权重 matrix:

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- For those 谁 are familiar with one-hot encoding, 这个 嵌入 层 approach above is essentially just 一个 more efficient way of implementing one-hot encoding followed by matrix multiplication in 一个 fully-connected 层, 哪个 is described in 这个 supplementary 代码 in [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)
- Because 这个 嵌入 层 is just 一个 more efficient 实现 那个 is equivalent to 这个 one-hot encoding 和 matrix-multiplication approach 它 can be seen as 一个 神经网络 层 那个 can be optimized via 反向传播

- To 转换 一个 词元 with id 3 into 一个 3-dimensional vector, 我们 do 这个 following:

In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- Note 那个 这个 above is 这个 4th row in 这个 `embedding_layer` 权重 matrix
- To embed all four `input_ids` values above, 我们 do

In [46]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 一个 嵌入 层 is essentially 一个 look-up operation:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **你 may be interested in 这个 bonus content comparing 嵌入 layers with regular linear layers: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 Encoding word positions

- 嵌入 层 转换 IDs into identical vector representations regardless of 哪里 they are located in 这个 输入 sequence:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- Positional embeddings are combined with 这个 词元 嵌入 vector to form 这个 输入 embeddings for 一个 大语言模型:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- 这个 BytePair encoder has 一个 vocabulary size of 50,257:
- Suppose 我们 want to encode 这个 输入 tokens into 一个 256-dimensional vector representation:

In [47]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 如果 我们 sample data from 这个 dataloader, 我们 embed 这个 tokens in each batch into 一个 256-dimensional vector
- 如果 我们 have 一个 批量大小 of 8 with 4 tokens each, 这个 results in 一个 8 x 4 x 256 tensor:

In [48]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [49]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & 执行 这个 following line to see 如何 这个 embeddings look like
# 打印(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2 uses absolute position embeddings, so 我们 just 创建 another 嵌入 层:

In [51]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & 执行 这个 following line to see 如何 这个 嵌入 层 weights look like
# 打印(pos_embedding_layer.权重)

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & 执行 这个 following line to see 如何 这个 embeddings look like
# 打印(pos_embeddings)

torch.Size([4, 256])


- To 创建 这个 输入 embeddings used in 一个 大语言模型, 我们 simply 添加 这个 词元 和 这个 positional embeddings:

In [53]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & 执行 这个 following line to see 如何 这个 embeddings look like
# 打印(input_embeddings)

torch.Size([8, 4, 256])


- In 这个 initial phase of 这个 输入 processing workflow, 这个 输入 text is segmented into separate tokens
- Following 这个 segmentation, these tokens are transformed into 词元 IDs based on 一个 predefined vocabulary:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# Summary 和 takeaways

See 这个 [./dataloader.ipynb](./dataloader.ipynb) 代码 笔记本, 哪个 is 一个 concise version of 这个 数据加载器 那个 我们 implemented in 这个 第 和 will need for 训练 这个 GPT 模型 in upcoming chapters.

See [./练习-解答.ipynb](./练习-解答.ipynb) for 这个 练习 解答.

See 这个 [Byte Pair Encoding (BPE) 分词器 From Scratch](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) 笔记本 如果 你 are interested in learning 如何 这个 GPT-2 分词器 can be implemented 和 trained from scratch.